# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
vacation_csv = "../output_data/cities.csv"

vacation_df =pd.read_csv(vacation_csv)
vacation_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


In [4]:
#vacation_df.dtypes

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# List for Data
lat = []
lng = []
locations = []
humidity = []

# Extracting Data from DF
lat = vacation_df["Lat"]
lng = vacation_df["Lng"]
humidity = vacation_df ["Humidity"]

for i in range(len(lat)):
    
    locations.append([lat[i], lng[i]])

In [6]:
#Config Gmaps

gmaps.configure(api_key = g_key)

fig = gmaps.figure()
fig

#Humidity Coordinates

humidity_locations = locations

heatmap_layer = gmaps.heatmap_layer(humidity_locations, weights = humidity, max_intensity= 75, point_radius = 5
)

# Add the layer to the map
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
new_vacation_df = pd.DataFrame(vacation_df, columns = ["City", "Cloudiness", "Max Temp", "Wind Speed", "Lat", "Lng",])
new_vacation_df

,City,Cloudiness,Max Temp,Wind Speed,Lat,Lng
0,ostrovnoy,2,37.50,7.16,68.05,39.51
1,mahebourg,75,78.80,11.41,-20.41,57.70
2,qaanaaq,25,22.20,2.37,77.48,-69.36
3,zhuhai,0,44.60,4.47,40.71,112.04
4,cape town,20,55.99,8.05,-33.93,18.42
...,...,...,...,...,...,...
543,yabrud,0,80.60,8.05,33.97,36.66
544,paraiso,5,84.20,16.11,24.01,-104.61
545,veraval,0,61.00,8.05,49.65,0.71
546,novyy urgal,100,49.74,2.68,51.07,132.56


In [8]:
# DROP NA values

vacation_NA_df = new_vacation_df.dropna()
vacation_NA_df


,City,Cloudiness,Max Temp,Wind Speed,Lat,Lng
0,ostrovnoy,2,37.50,7.16,68.05,39.51
1,mahebourg,75,78.80,11.41,-20.41,57.70
2,qaanaaq,25,22.20,2.37,77.48,-69.36
3,zhuhai,0,44.60,4.47,40.71,112.04
4,cape town,20,55.99,8.05,-33.93,18.42
...,...,...,...,...,...,...
543,yabrud,0,80.60,8.05,33.97,36.66
544,paraiso,5,84.20,16.11,24.01,-104.61
545,veraval,0,61.00,8.05,49.65,0.71
546,novyy urgal,100,49.74,2.68,51.07,132.56


In [9]:
# Dropping Values > 80

drop_80_df = vacation_NA_df.loc[vacation_NA_df["Max Temp"] <= 80]

# Dropping Values < 70

drop_70_df = drop_80_df.loc[drop_80_df["Max Temp"] > 70]

# Droppiung Cloudiness

no_clouds = drop_70_df.loc[drop_70_df["Cloudiness"] == 0]
no_clouds

# Dropping Wind Speed > 10 MPH

no_wind_df = no_clouds.loc[no_clouds["Wind Speed"] < 10 ]
no_wind_df.head()

,City,Cloudiness,Max Temp,Wind Speed,Lat,Lng
7,opuwo,0,72.06,5.44,-18.06,13.84
70,ponta do sol,0,73.68,5.32,-20.63,-46.00
88,cidreira,0,76.56,5.50,-30.17,-50.22
138,morondava,0,79.08,9.06,-20.30,44.28
139,vaini,0,78.90,1.59,15.34,74.49


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
# Pulling City, Lat and Lng for Dataframe

hotel_df = pd.DataFrame(no_wind_df, columns= ["City", "Lat", "Lng"])
hotel_df

# Creating "Hotel Name" column

hotel_df["Hotel Name"] = " "
hotel_df["Country"]= " "
hotel_df

,City,Lat,Lng,Hotel Name,Country
7,opuwo,-18.06,13.84,,
70,ponta do sol,-20.63,-46.00,,
88,cidreira,-30.17,-50.22,,
138,morondava,-20.30,44.28,,
139,vaini,15.34,74.49,,
176,nador,35.17,-2.93,,
297,mogok,22.92,96.51,,
444,birjand,32.86,59.22,,


In [11]:
target_coordinates = f"{hotel_df['Lat']},{hotel_df['Lng']}"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

# Parameters

params = {
    "location:": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# Base URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Response
response = requests.get(base_url, params=params)



In [12]:
# List for Hotel Results
hotel_name = []

params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key  
}

# Base URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


# Iterate from Hotel_DF
for index, row in hotel_df.iterrows():
    
    # get lat, lng from df
    
    lat = row["Lat"]
    lng = row["Lng"]
              

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    try:
#         hotel_df.iloc[index, 4] = name_address['results'][0]['name']
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)
              
    
              

In [13]:
hotel_df["Hotel Name"] = hotel_name
hotel_df.dropna

hotel_df

,City,Lat,Lng,Hotel Name,Country
7,opuwo,-18.06,13.84,Opuwo Country Lodge,
70,ponta do sol,-20.63,-46.00,Escarpas Resort,
88,cidreira,-30.17,-50.22,Hotel Castelo,
138,morondava,-20.30,44.28,Palissandre Cote Ouest resort & SPA,
139,vaini,15.34,74.49,Dandeli Dreams Jungle Resort,
176,nador,35.17,-2.93,"Hotel Marchica Lagoon Resort, Nador Morocco",
297,mogok,22.92,96.51,MOGOK HILL HOTEL,
444,birjand,32.86,59.22,هتل پرويز,


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))